In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer 
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from scipy.special import expit
from sklearn.metrics import classification_report

In [2]:
def sigmoid(x):
    return expit(x)

In [3]:
data = load_breast_cancer()
X_train,X_test,Y_train,Y_test=train_test_split(data.data,data.target,train_size=.75, random_state=0)

/home/sacha/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
X = normalize(X_train).transpose()
Y = Y_train.transpose()

In [149]:
w = np.random.randn(30, 1)*np.sqrt(1/(30-1))
b = np.array([0])
alpha = 0.01
beta = 0.9
betatwo = 0.999
eta = 10e-8
vdW = 0
vdB = 0
sdW = 0
sdB = 0
batch_size = 512

In [150]:
for i in range(1,30000):
    for t in range(0, len(X), batch_size):
                  
        #batches
        x_batch = X[:, t:t+batch_size]
        y_batch = Y[t:t+batch_size]
        
        #forward
        Z = np.dot(w.transpose(), x_batch) + b
        A = sigmoid(Z)
        L = np.sum(-(y_batch*np.log10(A)+(1-y_batch)*np.log10(1-A)))/len(A) # Binary Loss

        #backward (derivatives)
        dZ = (A-y_batch)

        #W
        dW = x_batch.dot(dZ.transpose())/len(dZ)
        vdW = beta*vdW + (1-beta)*dW # Momentum Terms
        vdWC = vdW/(1-beta**i) # Bias Correction
        sdW = betatwo*sdW + (1-betatwo)*dW**2 #RMS terms
        sdWC = sdW/(1-betatwo**i) # Bias Correction
        w = w - alpha*vdWC/(np.sqrt(sdWC) + eta) # Adam

        #B
        dB = np.sum(dZ)/len(dZ)
        vdB = beta*vdB + (1-beta)*dB # Momentum Terms
        vdBC = vdB/(1-beta**i) # Bias Correction
        sdB = betatwo*sdB + (1-betatwo)*dB**2 # RMS terms
        sdBC = sdB/(1-betatwo**i) # Bias Correction
        b = b - alpha*vdBC/(np.sqrt(sdBC) + eta) # Adam

        if(i%5000==0 or i==1):
            print('Logloss: {}'.format(L))

Logloss: 129.86535624860448
Logloss: 38.110681016328755
Logloss: 32.376539012018874
Logloss: 29.48868770298401
Logloss: 27.181557359530576
Logloss: 25.25348917299973


In [148]:
Z = np.dot(w.transpose(), normalize(X_test).transpose()) + b
A = sigmoid(Z)
A[A>0.5] = 1
A[A<=0.5] = 0
predictions = A
print(classification_report(Y_test, predictions.reshape(143,)))

             precision    recall  f1-score   support

          0       0.91      0.92      0.92        53
          1       0.96      0.94      0.95        90

avg / total       0.94      0.94      0.94       143

